In [58]:
import os
import numpy as np
import polyscope as ps
from skimage import measure
from scipy.spatial import distance
import scipy.linalg
from IPython.display import display
from functools import partial

def load_off_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Parse the vertices and faces from the OFF file
    num_vertices, num_faces, _ = map(int, lines[1].split())

    vertices = np.array([list(map(float, line.split())) for line in lines[2:2 + num_vertices]])
    faces = np.array([list(map(int, line.split()))[1:] for line in lines[2 + num_vertices:]])

    return vertices, faces



def polyharmonic(r, k = 3):
    return np.power(r, k)
    
def Wendland(r, beta=1):
    return (1/12) * np.power(np.maximum(1 - beta * r, 0), 3) * (1 - 3 * beta * r)

def biharmonic(r):
    return r




### ================================ GENERATE RBF CENTRES  ==========================================


def generate_rbf_centres_targets(surfacePoints, normals, indices, epsilon):
    if indices is None:
        return surfacePoints
    elif len(indices) > 0:
        perturbedSurfacePoints = surfacePoints[indices]
        indexedNormals = normals[indices]
    else:
        perturbedSurfacePoints = surfacePoints
        indexedNormals = normals
        
    
    RBFCentres = np.concatenate ((
        surfacePoints,
        perturbedSurfacePoints + epsilon * indexedNormals,
        perturbedSurfacePoints - epsilon * indexedNormals
    ))


    return RBFCentres

### ================================ RBF DISTANCE MATRICES ==========================================

def basic_RBF_matrix(surfacePoints, normals, kernel, epsilon):
    
    RBFCentres = generate_rbf_centres_targets(surfacePoints, normals, [], epsilon)

    assert kernel == polyharmonic or kernel == Wendland or kernel == biharmonic, "Invalid kernel function"


    RBFMatrix = np.zeros((RBFCentres.shape[0], surfacePoints.shape[0]))
    
    pairwise_distances = distance.cdist(RBFCentres, RBFCentres, 'euclidean')

    RBFMatrix = kernel(pairwise_distances)

    return RBFMatrix, RBFCentres

def reduced_RBF_matrix(surfacePoints, kernel, RBFCentreIndices, normals, epsilon):

    dataPoints = generate_rbf_centres_targets(surfacePoints, normals, [], epsilon)

    RBFCentres = generate_rbf_centres_targets(surfacePoints, normals, RBFCentreIndices, epsilon)

    RBFMatrix = np.zeros(
        (len(dataPoints), len(RBFCentres))
    )

    print (len(dataPoints), len(RBFCentres))
    
    pairwise_distances = distance.cdist( dataPoints, RBFCentres, 'euclidean' )

    RBFMatrix = kernel(pairwise_distances)

    return RBFMatrix, RBFCentres


### ================================ POLYNOMIAL MATRIX GENERATION ======================================

def generate_polynomial_matrix(RBFCentres, degree, SPACE_DIM = 3):
    # Generate all indices up to the degree
    indices = np.arange(degree + 1)
    
    # Generate all combinations of indices where the sum is less than or equal to the degree
    indices_combinations = np.array(np.meshgrid(*[indices] * SPACE_DIM)).T.reshape(-1, SPACE_DIM)
    indices_combinations = indices_combinations[np.sum(indices_combinations, axis=1) <= degree]
    
    # Initialize Q with the correct shape
    Q = np.zeros((RBFCentres.shape[0], indices_combinations.shape[0]))
    
    # Compute the values for Q using broadcasting
    RBFCentres_bc = RBFCentres[:, np.newaxis, :]  # Add new axis for broadcasting
    indices_combinations_bc = indices_combinations[np.newaxis, :, :]  # Add new axis for broadcasting
    
    Q = np.prod(np.power(RBFCentres_bc, indices_combinations_bc), axis=2)
    
    return Q


### ================================ RBF WEIGHTS COMPUTATION ======================================


def compute_basic_weights(surfacePoints, target_vals, kernel, normals, epsilon):
    RBFMatrix, RBFCentres = basic_RBF_matrix(surfacePoints, normals, kernel, epsilon)

    LU_factorization = scipy.linalg.lu_factor(RBFMatrix)

    weights = scipy.linalg.lu_solve(LU_factorization, target_vals)

    print (weights.shape)

    return weights, RBFCentres, []


def compute_polynomial_weights(surfacePoints, normals, target_vals, kernel, degree, epsilon):

    RBFMatrix, RBFCentres = basic_RBF_matrix(surfacePoints, normals, kernel, epsilon)

    Q = generate_polynomial_matrix(RBFCentres, degree)
    

    assert RBFMatrix.shape[0] == Q.shape[0], "Q: {} RBFMatrix: {}".format(Q.shape, RBFMatrix.shape)

    # new system of equations:
        
    # | A   Q | w = b
    # | Q^T 0 | a = 0
    
    n_terms = Q.shape[1]
    zeroes = np.zeros((n_terms, n_terms))


    M = np.block([
        [RBFMatrix,       Q],
        [Q.T,        zeroes]
    ])

    target_vals = np.concatenate((
        target_vals,
        np.zeros(n_terms)
    ) , axis=0)


    LU_factorization = scipy.linalg.lu_factor(M)

    Solution = scipy.linalg.lu_solve(LU_factorization, target_vals)


    weights = Solution[:RBFCentres.shape[0] ]
    a       = Solution[ RBFCentres.shape[0]:]
    
    return weights, RBFCentres, a


def compute_reduced_weights(surfacePoints, target_vals, kernel, RBFCentreIndices, normals, epsilon):

    
    RBFMatrix, RBFCentres = reduced_RBF_matrix(surfacePoints, kernel, RBFCentreIndices, normals, epsilon)
    ### System is over-determined, so we use least squares

    weights, _, _, _ = np.linalg.lstsq(RBFMatrix, target_vals, rcond=None)

    return weights, RBFCentres, []


### ================================ RBF EVALUATION ======================================


def compute_RBF_weights(inputPoints, inputNormals, RBFFunction, epsilon, RBFCentreIndices=[], useOffPoints=True,
                        sparsify=False, l=-1):
    SPACE_DIM = 3
    

    target_vals = np.repeat([0, epsilon, -epsilon], inputPoints.shape[0])


    use_polynomial = l != -1

    if useOffPoints:
        if use_polynomial:
            return compute_polynomial_weights(inputPoints, inputNormals, target_vals, RBFFunction, l, epsilon )

        if len( RBFCentreIndices ) == 0:

            return compute_basic_weights(inputPoints, target_vals, RBFFunction, inputNormals, epsilon)

        return compute_reduced_weights(inputPoints, target_vals, RBFFunction, RBFCentreIndices, inputNormals, epsilon)


    return compute_reduced_weights(inputPoints, target_vals, RBFFunction, None, inputNormals, epsilon)
    
    


def evaluate_RBF(xyz, centres, RBFFunction, weights, l=-1, a=[]):
    """
    Evaluate the RBF function at the given points.
    """

    ## F(x) = Q*a + A*w


    ### determine 

    A = distance.cdist(xyz, centres, 'euclidean')

    values = np.dot(RBFFunction(A), weights)

    use_polynomial = l != -1

    if not use_polynomial:
        return values

    Q = generate_polynomial_matrix(xyz, l)

    values = values + np.dot(Q, a)

    return values




inputPoints = np.array([
    [0, 0, 0],
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

inputNormals = np.array([
    [0, 0, 1],
    [0, 0, 1],
    [0, 0, 1],
    [0, 0, 1]
])

indices = []


weights, centres, a = compute_RBF_weights(inputPoints, inputNormals, polyharmonic, 1, useOffPoints=False, RBFCentreIndices=indices)
print (evaluate_RBF(inputPoints, centres, polyharmonic, weights, a=a))



12 4
[-0.22684659 -0.19886856 -0.19886856  0.07789328]


In [59]:

ps.init()

inputPointNormals, _ = load_off_file(os.path.join('..', 'data', 'bunny-500.off'))
inputPoints = inputPointNormals[:, 0:3]
inputNormals = inputPointNormals[:, 3:6]

# normalizing point cloud to be centered on [0,0,0] and between [-0.9, 0.9]
inputPoints -= np.mean(inputPoints, axis=0)
min_coords = np.min(inputPoints, axis=0)
max_coords = np.max(inputPoints, axis=0)
scale_factor = 0.9 / np.max(np.abs(inputPoints))
inputPoints = inputPoints * scale_factor

ps_cloud = ps.register_point_cloud("Input points", inputPoints)
ps_cloud.add_vector_quantity("Input Normals", inputNormals)



# Parameters
gridExtent = 1 #the dimensions of the evaluation grid for marching cubes
res = 50 #the resolution of the grid (number of nodes)

# Generating and registering the grid
gridDims = (res, res, res)
bound_low = (-gridExtent, -gridExtent, -gridExtent)
bound_high = (gridExtent, gridExtent, gridExtent)
ps_grid = ps.register_volume_grid("Sampled Grid", gridDims, bound_low, bound_high)

X, Y, Z = np.meshgrid(np.linspace(-gridExtent, gridExtent, res),
                        np.linspace(-gridExtent, gridExtent, res),
                        np.linspace(-gridExtent, gridExtent, res), indexing='ij')

#the list of points to be fed into evaluate_RBF
xyz = np.column_stack((X.flatten(), Y.flatten(), Z.flatten()))

##########################
## you code of computation and evaluation goes here
##
##


RBFFUNCTION, L, OFFPOINTS = Wendland, 2, True

RBFIndices = np.random.randint(0, inputPoints.shape[0], 200)


w, AllSurfacePoints, a = compute_RBF_weights(
    inputPoints, inputNormals, RBFFUNCTION, 0.01,
    useOffPoints=OFFPOINTS, sparsify=False, l=L,
    RBFCentreIndices=RBFIndices
)


print(w.shape)


RBFValues = evaluate_RBF(xyz, AllSurfacePoints, RBFFUNCTION, w, a=a, l=L)




# RBFValues = xyz[:,0]**2+xyz[:,1]**2+xyz[:,2]**2-0.5 #stub sphere


##
##
#########################

#fitting to grid shape again


(1506,)


In [60]:
RBFValues = np.reshape(RBFValues, X.shape)

# Registering the grid representing the implicit function
ps_grid.add_scalar_quantity("Implicit Function", RBFValues, defined_on='nodes',
                            datatype="standard", enabled=True)



# Computing marching cubes and realigning result to sit on point cloud exactly
vertices, faces, _, _ = measure.marching_cubes(RBFValues, spacing=(
    2.0 * gridExtent / float(res - 1), 2.0 * gridExtent / float(res - 1), 2.0 * gridExtent / float(res - 1)),
                                                level=0.0)
vertices -= gridExtent
ps.register_surface_mesh("Marching-Cubes Surface", vertices, faces)

ps.show()
